**The same task as Lab 1 except using `Word Embeddings OR RNNs`**

**Watch out!<br>
Preproceesing here is a a bit different<br>
Just take the cleaned data from the previous lab and cont...**


**Dataset**
labeled datasset collected from twitter

**Objective**
classify tweets containing hate speech from other tweets.
0 -> no hate speech
1 -> contains hate speech

**Total Estimated Time = 60 Mins**

### Import Libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions# load spacy model, can be "en_core_web_sm" as well
nlp = spacy.load('en_core_web_md')

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re

pd.set_option('display.max_colwidth', 100000)

### Load Dataset

In [3]:
Data_set = pd.read_csv("dataset.csv" , index_col="id")

### Preprocessing


In [4]:
duplicated_df = Data_set[Data_set[["tweet" ,"label"]].duplicated()]

In [5]:
Data_set = Data_set.drop(duplicated_df.index)

In [6]:
Data_set.head()

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
3,0,bihday your majesty
4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
5,0,factsguide: society now #motivation


In [7]:
def process_tweet(tweet):
    return " ".join(re.sub("([\@|\#][A-Za-z0-9]+)|([^0-9A-Za-z \t]) | r's+[a-zA-Z]s+' ", " ",tweet.lower()).split())

def remove_single_char(text):
    return re.sub(r's+[a-zA-Z]s+', ' ', text)

def text_preprocessing(text): 
    
    text = process_tweet(text)
    
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")

    text = text.strip()
    text =  " ".join(text.split())

    text = unidecode.unidecode(text)

    text = contractions.fix(text)

    text = text.lower()
    

    doc = nlp(text) 
    clean_text = []
    
    for token in doc:
        flag = True
        edit = token.text
        
        if token.is_stop and token.pos_ != 'NUM': 
            flag = False
        if token.pos_ == 'PUNCT' and flag == True: 
            flag = False
        if token.pos_ == 'SYM' and flag == True: 
            flag = False
        if (token.pos_ == 'NUM' or token.text.isnumeric()) \
        and flag == True:
            flag = False
        if token.pos_ == 'NUM' and flag == True:
            edit = w2n.word_to_num(token.text)
        elif token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        if edit != "" and flag == True:
            clean_text.append(edit) 
        
    return clean_text

In [8]:
Data_set["tweet"] = Data_set["tweet"].apply(lambda x:' '.join(text_preprocessing(x)))

C:\Users\Abdelrahman\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [85]:
# Data = ' '.join(text_preprocessing(Data_set["tweet"].iloc[9]))
# remove_single_char(Data)

In [9]:
Data_set.head(5)

,label,tweet
id,,
1,0,father dysfunctional selfish drag kid dysfunction
2,0,thank credit use offer wheelchair van pdx
3,0,bihday majesty
4,0,love time urd+- dddd d|d|d
5,0,factsguide society


### Modelling
**Use any Advanced technique such as: word2vec, glove, RNNs ... etc**

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=15000 ,oov_token='UNK')
tok.fit_on_texts(Data_set["tweet"])

In [11]:
print(tok.word_index['UNK'])
print(len(tok.word_index))
length_of_word_index = len(tok.word_index)

1
19853


In [13]:
from sklearn.model_selection import train_test_split
target=Data_set["label"]
train_words, validation_words ,train_target ,validation_target = train_test_split(Data_set['tweet'],target,test_size=0.2)

In [14]:
train_words_binary = tok.texts_to_matrix(train_words, mode='binary')
validation_words_binary = tok.texts_to_matrix(validation_words, mode='binary')

In [15]:
print(train_words.shape , train_target.shape)
print(validation_words.shape , validation_target.shape)

(23624,) (23624,)
(5906,) (5906,)


In [16]:
from keras import layers,models

model=models.Sequential()
model.add(layers.Dense(128,activation='relu',input_shape=(15000,)))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [17]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras
model.compile(optimizer= RMSprop(lr=2e-5),
              loss= "binary_crossentropy",
              metrics= ["binary_accuracy"])

model.fit(train_words_binary,train_target,epochs=10,validation_data=(validation_words_binary,validation_target))

C:\Users\Abdelrahman\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
739/739 [==============================] - 25s 32ms/step - loss: 0.6223 - binary_accuracy: 0.9229 - val_loss: 0.5348 - val_binary_accuracy: 0.9368
Epoch 2/10
739/739 [==============================] - 21s 28ms/step - loss: 0.4208 - binary_accuracy: 0.9306 - val_loss: 0.3086 - val_binary_accuracy: 0.9368
Epoch 3/10
739/739 [==============================] - 21s 28ms/step - loss: 0.2596 - binary_accuracy: 0.9306 - val_loss: 0.2272 - val_binary_accuracy: 0.9368
Epoch 4/10
739/739 [==============================] - 21s 29ms/step - loss: 0.2312 - binary_accuracy: 0.9306 - val_loss: 0.2210 - val_binary_accuracy: 0.9368
Epoch 5/10
739/739 [==============================] - 21s 29ms/step - loss: 0.2257 - binary_accuracy: 0.9306 - val_loss: 0.2182 - val_binary_accuracy: 0.9368
Epoch 6/10
739/739 [==============================] - 21s 28ms/step - loss: 0.2216 - binary_accuracy: 0.9306 - val_loss: 0.2161 - val_binary_accuracy: 0.9368
Epoch 7/10
739/739 [==============================] 

#### Evaluation

#### Done!